In [ ]:
!python -V

In [ ]:
# Uncomment to install required packages
#!conda install -c conda-forge -c pytorch u8darts-all ipywidgets psutil gputil tabulate tensorflow -y

In [ ]:
import os
import time
import psutil
import GPUtil
import platform
import tabulate
import datetime

from tabulate import tabulate

print("="*120)
print("-"*48, "Uptime Information", "-"*52)
print(f"Last boot:    ", datetime.datetime.fromtimestamp(psutil.boot_time()))
print(f"Current time: ", datetime.datetime.now())
uptimeSeconds = time.time() - psutil.boot_time()
uptime = time.strftime('%dd %Hh %Mm', time.gmtime(uptimeSeconds))
print(f"Current time: ", uptime)
#print(f"Uptime: ", uptime)
print("-"*48, "System Information", "-"*52)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print("-"*48, "Hardware Information", "-"*50)
print(f"Processor: {uname.processor}")
print("Physical cores:", psutil.cpu_count(logical=False), " - Total cores:", psutil.cpu_count(logical=True))
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.0f}Mhz", f" - Current Frequency: {cpufreq.current:.0f}Mhz")
svmem = psutil.virtual_memory()
print("Total memory (GB):{:.0f}".format(svmem.total/1e9))
print("-"*48, "GPU Information", "-"*55)
gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    gpu_id = gpu.id
    gpu_name = gpu.name
    gpu_load = f"{gpu.load*100}%"
    gpu_free_memory = f"{gpu.memoryFree}MB"
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))
print("="*120)

In [ ]:
import time
start = time.time()

#Common module parameters:
NUMPY_RANDOM_SEED=10
figsize = (9, 6)

#Train validate test split (70%/15%/15%). Test split is inferred from test and validate values.
train_size = 0.70
valid_size=0.15


LAZY_WALK_LAG=1 # # of periods (hours) to compute metrics for a lazy walk baseline scenario.

MODEL_NAME_RNN='2v1_ALLDNNs-EPOCHS_1000-48-2-BASE_c1'
MODEL_NAME_LSTM='2v1_ALLDNNs-EPOCHS_100-48-2-BASE_c2'
MODEL_NAME_TCN='2v1_ALLDNNs-EPOCHS_100-48-2-BASE_c3'
MODEL_NAME_GRU='2v1_ALLDNNs-EPOCHS_100-48-2-BASE_c4'
MODEL_NAME_NBEATS='2v1_ALLDNNs-EPOCHS_100-48-2-BASE_c5'
MODEL_NAME_TFT='2v1_ALLDNNs-EPOCHS_100-48-2-BASE_c6'

# FOR MACOS "/darts_logs/" OR "\\darts_logs\\" FOR WIN - CHOOSE ACCORDINGLY:
LOG_PATH="/darts_logs/" 
TENSORBOARD_LOG_PATH_RNN = ((os.getcwd() + LOG_PATH + MODEL_NAME_RNN))
TENSORBOARD_LOG_PATH_LSTM = ((os.getcwd() + LOG_PATH + MODEL_NAME_LSTM))
TENSORBOARD_LOG_PATH_TCN = ((os.getcwd() + LOG_PATH + MODEL_NAME_TCN))
TENSORBOARD_LOG_PATH_GRU = ((os.getcwd() + LOG_PATH + MODEL_NAME_GRU))
TENSORBOARD_LOG_PATH_NBEATS = ((os.getcwd() + LOG_PATH + MODEL_NAME_NBEATS))
TENSORBOARD_LOG_PATH_TFT = ((os.getcwd() + LOG_PATH + MODEL_NAME_TFT))

#Deep Neural Network parameters:
RANDOM_STATE=31
INPUT_CHUNK_LENGTH = 48 #periods = hours
OUTPUT_CHUNK_LENGTH = 2
FORECAST_HORIZON = 2 #periods = hours
HIST_FOR_TIME_START = '2021-10-17T11:00:00' #must be minimum (FORECAST_HORIZON +1) periods ahead of the test start series

N_EPOCHS_RNN=100
N_EPOCHS_LSTM=100
N_EPOCHS_TCN=100
N_EPOCHS_GRU=100
N_EPOCHS_NBEATS=100
N_EPOCHS_TFT=100

In [ ]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm_notebook as tqdm


import matplotlib.pyplot as plt
import torch

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel, RNNModel, TCNModel, BlockRNNModel, NBEATSModel, ARIMA, Prophet
from darts.metrics import mape, rmse, r2_score, mae, mse

import warnings
warnings.filterwarnings("ignore")

import logging
logging.disable(logging.CRITICAL)

%matplotlib inline

In [ ]:
#DARTS version 0.17.1
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

In [ ]:
#CONSTANT VALUE TEST TIMESERIES

# np.random.seed(NUMPY_RANDOM_SEED)
# series_constant = tg.constant_timeseries(
#     value=1.0,
#     start=pd.to_datetime(TS_START_DATETIME),
#     end=pd.to_datetime(TS_END_DATETIME),
# #   length=TS_LENGTH,
#     freq=TS_FREQUENCY,
#     column_name='c',
#     )

In [ ]:
# #RANDOM WALK TEST TIMESERIES - Creates a random walk univariate TimeSeries, where each step is obtained by sampling a
# #gaussian distribution with mean mean and standard deviation std.

# np.random.seed(NUMPY_RANDOM_SEED)
# series_random_walk = tg.random_walk_timeseries(
#     mean=0.0,
#     std=1.0,
#     start=pd.to_datetime(TS_START_DATETIME),
#     end=pd.to_datetime(TS_END_DATETIME),
# #   length=TS_LENGTH,
#     freq=TS_FREQUENCY,
#     column_name='c',
#     )

In [ ]:
df = pd.read_csv(r"C:\Users\User\Downloads\price-ohlc-eth-1h-final.csv", parse_dates=["timestamp"])
df['timestamp'] = pd.to_datetime(df.timestamp).dt.tz_localize(None)
nof_missing_values_o = sum(np.isnan(df['o']))
nof_missing_values_h = sum(np.isnan(df['h']))
nof_missing_values_l = sum(np.isnan(df['l']))
nof_missing_values_c = sum(np.isnan(df['c']))

df = df.fillna(method="backfill")
nof_missing_values = sum(np.isnan(df['o']))

df = df.fillna(method="backfill")
nof_missing_values = sum(np.isnan(df['h']))

df = df.fillna(method="backfill")
nof_missing_values = sum(np.isnan(df['l']))

df = df.fillna(method="backfill")
nof_missing_values = sum(np.isnan(df['c']))

train_index = int(len(df)*train_size)

df_train = df[0:train_index]
df_rem = df[train_index:]

valid_index = int(len(df)*valid_size)

df_val = df[train_index:train_index+valid_index]
df_test = df[train_index+valid_index:]

series = TimeSeries.from_dataframe(df, time_col='timestamp', value_cols=['c']).astype(np.float32)
series_covariates = TimeSeries.from_dataframe(df, time_col='timestamp', value_cols=['h','l','o']).astype(np.float32)
train = TimeSeries.from_dataframe(df_train, time_col='timestamp', value_cols=['c']).astype(np.float32)
train_covariates = TimeSeries.from_dataframe(df_train, time_col='timestamp', value_cols=['h','l','o']).astype(np.float32)
val = TimeSeries.from_dataframe(df_val, time_col='timestamp', value_cols=['c']).astype(np.float32)
val_covariates = TimeSeries.from_dataframe(df_val, time_col='timestamp', value_cols=['h','l','o']).astype(np.float32)
test = TimeSeries.from_dataframe(df_test, time_col='timestamp', value_cols=['c']).astype(np.float32)
test_covariates = TimeSeries.from_dataframe(df_test, time_col='timestamp', value_cols=['h','l','o']).astype(np.float32)

In [ ]:
# from darts.utils.statistics import check_seasonality, plot_acf

# print('Seasonality: ',check_seasonality(series, max_lag=(len(series)-1)))
# plot_acf(series, 1, max_lag=(len(series)-1))

In [ ]:
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
test_transformed = transformer.transform(test)
train_covariates_transformed = transformer.fit_transform(train_covariates)
val_covariates_transformed = transformer.transform(val_covariates)
test_covariates_transformed = transformer.transform(test_covariates)

In [ ]:
train.plot(label="Train set")
val.plot(label="Validate set")
test.plot(label="Test set")
plt.legend(
           borderpad=2,
           labelspacing=1.0,
           title_fontsize='large',
           shadow=False,
           frameon=False,
           loc='best'  
          )
plt.ylabel('Ethereum Close value (USD)')
plt.title('Ethereum dataset - train, validate and test series',fontsize=13)

In [ ]:
print("="*72)
print('Dataset type',' -      Start time    -', '     End time','    -Samples', '- %split')
print("-"*72)
print('Full dataset: ', series.start_time(),'', series.end_time(),'', len(series),'     {:.0f}'.format((len(series)/len(series)*100)))
print('Train set:    ', train.start_time(),'', train.end_time(),'', len(train),'      {:.0f}'.format((len(train)/len(series)*100)))
print('Validate set: ', val.start_time(),'', val.end_time(),' ', len(val),'      {:.0f}'.format((len(val)/len(series)*100)))
print('Test set:     ', test.start_time(),'', test.end_time(),' ', len(test),'      {:.0f}'.format((len(test)/len(series)*100)))
print("="*72)

In [ ]:
tic_prep = time.time()
prep_elapsed_time = tic_prep - start
print(prep_elapsed_time,'seconds')

In [ ]:
rnn_model = RNNModel(
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=OUTPUT_CHUNK_LENGTH,
    model="RNN",
    hidden_dim=30,
    n_rnn_layers=1,
    loss_fn=torch.nn.MSELoss(),
    dropout=0.1,
    batch_size=16,
    n_epochs=N_EPOCHS_RNN,
    optimizer_cls=torch.optim.Adam,
    optimizer_kwargs={"lr": 1e-3},
    model_name=MODEL_NAME_RNN,
    log_tensorboard=True,
    random_state=RANDOM_STATE,
    force_reset=True,
    save_checkpoints=True,
    pl_trainer_kwargs={
        'accelerator': 'gpu', 'gpus': [0],
        'progress_bar_refresh_rate': True,
        'enable_model_summary': True,
#        'auto_scale_batch_size': True,
#        'auto_lr_find': True,
    },
    show_warnings=True,
)

rnn_model.fit(
    train_transformed,
    val_series=val_transformed,
    future_covariates=train_covariates_transformed,
    val_future_covariates=val_covariates_transformed,
    verbose=True,
)

In [ ]:
best_rnn_model = RNNModel.load_from_checkpoint(model_name=MODEL_NAME_RNN, best=True)

In [ ]:
backtest_series_rnn = best_rnn_model.historical_forecasts(
    series=test_transformed,
    future_covariates=test_covariates_transformed,
    start=pd.Timestamp(HIST_FOR_TIME_START),
    forecast_horizon=FORECAST_HORIZON,
    stride=1,
    retrain=False,
    verbose=False,
)

In [ ]:
tic_rnn = time.time()
rnn_elapsed_time = tic_rnn - tic_prep
print(rnn_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("RNN:           {:.5f}".format(mse(backtest_series_rnn, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_rnn, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_rnn, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_rnn, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_rnn, test_transformed))
)

In [ ]:
# !tensorboard dev upload --logdir={TENSORBOARD_LOG_PATH_RNN} \
#                         --name={MODEL_NAME_RNN} \
#                         --one_shot

In [ ]:
lstm_model = RNNModel(
    model="LSTM",
    hidden_dim=30,
    dropout=0.1,
    batch_size=16,
    n_epochs=N_EPOCHS_LSTM,
    optimizer_kwargs={"lr": 1e-3},
    model_name=MODEL_NAME_LSTM,
    log_tensorboard=True,
    random_state=RANDOM_STATE,
    training_length=INPUT_CHUNK_LENGTH,
    input_chunk_length=INPUT_CHUNK_LENGTH,
    force_reset=True,
    save_checkpoints=True,
    torch_device_str='cuda:0',
)

lstm_model.fit(
    train_transformed,
    val_series=val_transformed,
    future_covariates=train_covariates_transformed,
    val_future_covariates=val_covariates_transformed,
    verbose=True,
)

In [ ]:
best_lstm_model = RNNModel.load_from_checkpoint(model_name=MODEL_NAME_LSTM, best=True)

In [ ]:
backtest_series_lstm = best_lstm_model.historical_forecasts(
    series=test_transformed,
    future_covariates=test_covariates_transformed,
    start=pd.Timestamp(HIST_FOR_TIME_START),
    forecast_horizon=FORECAST_HORIZON,
    stride=1,
    retrain=False,
    verbose=False,
)

In [ ]:
tic_lstm = time.time()
lstm_elapsed_time = tic_lstm - tic_rnn
print(lstm_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("LSTM:          {:.5f}".format(mse(backtest_series_lstm, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_lstm, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_lstm, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_lstm, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_lstm, test_transformed))
)

In [ ]:
tcn_model = TCNModel(
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=FORECAST_HORIZON,
    n_epochs=N_EPOCHS_TCN,
    dropout=0.1,
#    num_layers= 10, #not defined prior to this version
    optimizer_kwargs={"lr": 1e-3},
    dilation_base=2,
    weight_norm=True,
    kernel_size=5,
    num_filters=3,
    random_state=RANDOM_STATE,
    force_reset=True,
    model_name=MODEL_NAME_TCN,
    log_tensorboard=True,
    save_checkpoints=True,
    torch_device_str='cuda:0',
)

tcn_model.fit(
    series=train_transformed,
    val_series=val_transformed,
#    future_covariates=train_covariates_transformed,
#    val_future_covariates=val_covariates_transformed,
    past_covariates=train_covariates_transformed,
    val_past_covariates=val_covariates_transformed,
    verbose=True,
)

In [ ]:
best_tcn_model = TCNModel.load_from_checkpoint(model_name=MODEL_NAME_TCN, best=True)

In [ ]:
backtest_series_tcn = best_tcn_model.historical_forecasts(
    series=test_transformed,
    past_covariates=test_covariates_transformed,
    start=pd.Timestamp(HIST_FOR_TIME_START),            
    forecast_horizon=FORECAST_HORIZON,
    stride=1,
    retrain=False,
    verbose=False,
)

In [ ]:
tic_tcn = time.time()
tcn_elapsed_time = tic_tcn - tic_lstm
print(tcn_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("TCN:           {:.5f}".format(mse(backtest_series_tcn, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_tcn, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_tcn, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_tcn, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_tcn, test_transformed))
)

In [ ]:
gru_model = BlockRNNModel(
    model="GRU",
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=FORECAST_HORIZON,
    hidden_size=30,
    n_rnn_layers=1,
    batch_size=16,
    n_epochs=N_EPOCHS_GRU,
    dropout=0.1,
    model_name=MODEL_NAME_GRU,
    nr_epochs_val_period=1,
    optimizer_kwargs={"lr": 1e-3},
    log_tensorboard=True,
    random_state=RANDOM_STATE,
    force_reset=True,
    save_checkpoints=True,
    torch_device_str='cuda:0',
)

gru_model.fit(
    series=train_transformed,
    val_series=val_transformed,
#     future_covariates=train_covariates_transformed,
#     val_future_covariates=val_covariates_transformed,
    past_covariates=train_covariates_transformed,
    val_past_covariates=val_covariates_transformed,
    verbose=True,
)

In [ ]:
best_gru_model = BlockRNNModel.load_from_checkpoint(model_name=MODEL_NAME_GRU, best=True)

In [ ]:
backtest_series_gru = best_gru_model.historical_forecasts(
    series=test_transformed,
    past_covariates=test_covariates_transformed,
    start=pd.Timestamp(HIST_FOR_TIME_START),
    forecast_horizon=FORECAST_HORIZON,
    stride=1,
    retrain=False,
    verbose=False,
)

In [ ]:
tic_gru = time.time()
gru_elapsed_time = tic_gru - tic_tcn
print(gru_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("GRU:          {:.5f}".format(mse(backtest_series_gru, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_gru, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_gru, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_gru, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_gru, test_transformed))
)

In [ ]:
nbeats_model = NBEATSModel(
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=FORECAST_HORIZON,
    generic_architecture=True,
    num_stacks=10,
    num_blocks=1,
    num_layers=4,
    layer_widths=512,
    n_epochs=N_EPOCHS_NBEATS,
    nr_epochs_val_period=1,
    batch_size=16,
    model_name=MODEL_NAME_NBEATS,
    save_checkpoints=True,
    random_state=RANDOM_STATE,
    force_reset=True,
    log_tensorboard=True,
    torch_device_str='cuda:0',
)

nbeats_model.fit(
    series=train_transformed,
    val_series=val_transformed,
#    future_covariates=train_covariates_transformed,
#    val_future_covariates=val_covariates_transformed,
    past_covariates=train_covariates_transformed,
    val_past_covariates=val_covariates_transformed,
    verbose=True,
)

In [ ]:
best_nbeats_model = NBEATSModel.load_from_checkpoint(model_name=MODEL_NAME_NBEATS, best=True)

In [ ]:
backtest_series_nbeats = best_nbeats_model.historical_forecasts(
    series=test_transformed,
    past_covariates=test_covariates_transformed,
    start=pd.Timestamp(HIST_FOR_TIME_START),
    forecast_horizon=FORECAST_HORIZON,
    stride=1,
    retrain=False,
    verbose=False,
)

In [ ]:
tic_nbeats = time.time()
nbeats_elapsed_time = tic_nbeats - tic_gru
print(nbeats_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("NBEATS:       {:.5f}".format(mse(backtest_series_nbeats, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_nbeats, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_nbeats, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_nbeats, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_nbeats, test_transformed))
)

In [ ]:
tft_model = TFTModel(
    input_chunk_length=INPUT_CHUNK_LENGTH,
    output_chunk_length=OUTPUT_CHUNK_LENGTH,
    hidden_size=32,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    n_epochs=N_EPOCHS_TFT,
    likelihood=None,
    loss_fn=torch.nn.MSELoss(),
    random_state=RANDOM_STATE,
    add_relative_index=True,
    force_reset=True,
    model_name=MODEL_NAME_TFT,
    save_checkpoints=True,
    log_tensorboard=True,
    torch_device_str='cuda:0',
)

tft_model.fit(
    train_transformed,
    val_series=val_transformed,
    past_covariates=train_covariates_transformed,
    val_past_covariates=val_covariates_transformed,
    verbose=True
)

In [ ]:
best_tft_model = TFTModel.load_from_checkpoint(model_name=MODEL_NAME_TFT, best=True)

In [ ]:
backtest_series_tft = best_tft_model.historical_forecasts(
    series=test_transformed,
    start=pd.Timestamp(HIST_FOR_TIME_START),
    past_covariates=test_covariates_transformed,
    forecast_horizon=FORECAST_HORIZON,
    stride=1,
    retrain=False,
    verbose=False,
)

In [ ]:
tic_tft = time.time()
tft_elapsed_time = tic_tft - tic_nbeats
print(tft_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("TFT:          {:.5f}".format(mse(backtest_series_tft, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_tft, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_tft, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_tft, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_tft, test_transformed))
)

In [ ]:
arima_model = ARIMA()
arima_model.fit(train_transformed)

In [ ]:
backtest_series_arima = arima_model.historical_forecasts(
   series=test_transformed,
   start=pd.Timestamp(HIST_FOR_TIME_START),
   forecast_horizon=FORECAST_HORIZON,
   stride=1,
   retrain=True,
   verbose=True,
)

In [ ]:
tic_arima = time.time()
arima_elapsed_time = tic_arima - tic_tft
print(arima_elapsed_time,'seconds')

In [ ]:
print("="*63)
print("---  Backtest metric results - Test set with best model     ---")
print("-"*63)
print("Model-Metrics    MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("ARIMA:        {:.5f}".format(mse(backtest_series_arima, test_transformed)),
     "  {:.4f}".format(mae(backtest_series_arima, test_transformed)),
     "  {:.4f}".format(rmse(backtest_series_arima, test_transformed)),
     "  {:.4f}".format(r2_score(backtest_series_arima, test_transformed)),
     "  {:.3f}%".format(mape(backtest_series_arima, test_transformed))
)

In [ ]:
test_transformed.plot(label="Ground Truth")
backtest_series_rnn.plot(label="RNN", color='green')
backtest_series_lstm.plot(label="LSTM", color='magenta')
backtest_series_tcn.plot(label="TCN", color='cyan')
backtest_series_gru.plot(label="GRU", color='red')
backtest_series_nbeats.plot(label="NBEATS", color='violet')
backtest_series_tft.plot(label="TFT")
backtest_series_arima.plot(label="ARIMA")

figsize = (36, 24)
plt.legend(
           borderpad=0,
           labelspacing=0.4,
           title_fontsize='large',
           shadow=False,
           frameon=False,
           loc='best'  
          )
plt.ylabel('Normalized ETH close value')

In [ ]:
print("="*33)
print('---  Tempo de Processamento   ---')
print("-"*33)
#print('1. Data handling:    {:.1f}'.format((prep_elapsed_time)),' seconds')
print('A. Modelo ARIMA:    {:.1f}'.format((arima_elapsed_time/60)),' minutes')
print("-"*33)
print('B. Modelo RNN:      {:.1f}'.format((rnn_elapsed_time)/60),' minutos')
print('C. Modelo LSTM:     {:.1f}'.format((lstm_elapsed_time)/60),' minutos')
print('D. Modelo TCN:      {:.1f}'.format((tcn_elapsed_time)/60),' minutos')
print('E. Modelo GRU:      {:.1f}'.format((gru_elapsed_time)/60),' minutos')
print('F. Modelo NBEATS:   {:.1f}'.format((nbeats_elapsed_time)/60), ' minutos')
print('G. Modelo TFT:      {:.1f}'.format((tft_elapsed_time)/60),' minutos')

print("="*33)

In [ ]:
print("="*63)
print("---         Resultado Backtest - Conjunto de teste         ---")
print("-"*63)
print("Modelo-Metrica   MSE", "      MAE ","   RMSE", "     R2      MAPE")
print("-"*63)
print("ARIMA:         {:.5f}".format(mse(backtest_series_arima, test_transformed)),
     "  {:.4f}".format(mae(backtest_series_arima, test_transformed)),
     "  {:.4f}".format(rmse(backtest_series_arima, test_transformed)),
     "  {:.4f}".format(r2_score(backtest_series_arima, test_transformed)),
     "  {:.3f}%".format(mape(backtest_series_arima, test_transformed))
)
print("-"*63)
print("RNN:           {:.5f}".format(mse(backtest_series_rnn, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_rnn, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_rnn, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_rnn, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_rnn, test_transformed))
)
print("LSTM:          {:.5f}".format(mse(backtest_series_lstm, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_lstm, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_lstm, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_lstm, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_lstm, test_transformed))
)
print("TCN:           {:.5f}".format(mse(backtest_series_tcn, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_tcn, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_tcn, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_tcn, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_tcn, test_transformed))
)
print("GRU:           {:.5f}".format(mse(backtest_series_gru, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_gru, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_gru, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_gru, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_gru, test_transformed))
)
print("NBEATS:        {:.5f}".format(mse(backtest_series_nbeats, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_nbeats, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_nbeats, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_nbeats, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_nbeats, test_transformed))
)
print("TFT:           {:.5f}".format(mse(backtest_series_tft, test_transformed)),
      "  {:.4f}".format(mae(backtest_series_tft, test_transformed)),
      "  {:.4f}".format(rmse(backtest_series_tft, test_transformed)),
      "  {:.4f}".format(r2_score(backtest_series_tft, test_transformed)),
      "  {:.3f}%".format(mape(backtest_series_tft, test_transformed))
)
#print("ARIMA:         {:.5f}".format(mse(backtest_series_arima, test_transformed)),
#      "  {:.4f}".format(mae(backtest_series_arima, test_transformed)),
#      "  {:.4f}".format(rmse(backtest_series_arima, test_transformed)),
#      "  {:.4f}".format(r2_score(backtest_series_arima, test_transformed)),
#      "  {:.3f}%".format(mape(backtest_series_arima, test_transformed))
#)
print("="*63)

In [ ]:
print("="*47)
print('-----  DNN Model Execution Information  -------')
print("-"*47)
print('1. Start time:            ',time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start)))
print('2. End time:              ',time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tic_arima)))
print('3. Total elapsed time:              {:.1f}'.format((tic_arima - start)/3600),' hours')
print("="*47)